In [1]:
from flask import Flask, request, send_file
import os
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import pandas as pd
import pyperclip

def efetuar_login(wait):
    # acessar icone do usuario
    try:
        icon_user = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="hdr-c"]/nav/div/a[3]/i')))
        icon_user.click()

        # acessar o login
        button_login = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="user-menu"]/div[2]/div[1]/a[1]')))
        button_login.click()

        # preencher login, senha e entrar
        input_user = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="username"]')))
        input_user.send_keys('mauriciopfleger')
        input_pass = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="password"]')))
        input_pass.send_keys('Jaqueta1!@')
        button_entrar = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="allCnt"]/main/app-authentication/app-signin/div/form/button')))
        button_entrar.click()
        print('realizado login')
    except:
        print('logado')

def apostar(wait, valor_aposta, jogo_unico):
    apostas = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="bslpb"]')))
    apostas.click()
    
    if jogo_unico:                                  
        valor = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="single-betslip"]/div[1]/app-coupon/div/div[2]/div[2]')))
    else:                                               
        valor = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="multi-betslip"]/div[2]/div[2]/div[1]/div/div/div[2]')))
    valor.click()  

    for num in valor_aposta:
        vnum = num
        if num == '0':
            vnum = '10'
        valor_num = wait.until(EC.element_to_be_clickable((By.XPATH, f'//*[@id="cNumpad"]/app-custom-numpad/div[3]/button[{vnum}]')))
        valor_num.click()
    
    enter = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="cNumpad"]/app-custom-numpad/div[3]/button[13]')))
    enter.click()
    try:
        if jogo_unico:
            aceitar = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="single-betslip"]/div[2]/div[2]/div[2]/a')))
        else:     
            aceitar = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="multi-betslip"]/div[2]/div[2]/div[2]/a')))
        if (aceitar.text.strip() == 'Aceitar todas as alterações'):
            aceitar.click()
    except:
        print('direto')
                                                                
    apostar = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="betslip-placebet-btn"]')))
    apostar.click()
    voltar = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="preview-betslip"]/div[3]/a[1]')))
    voltar.click()

erro = False

def caputrar_esportedasorte():
    # Especifica o caminho do arquivo que deseja excluir
    caminho_arquivo = "partidas.csv"

    # Verifica se o arquivo existe antes de excluí-lo
    if os.path.exists(caminho_arquivo):
        os.remove(caminho_arquivo)
        print(f"O arquivo {caminho_arquivo} foi excluído com sucesso")
    else:
        print(f"O arquivo {caminho_arquivo} não existe")

    chrome_options = Options()                   
    chrome_options.add_argument("--user-data-dir=C:\\Users\\mauri\\AppData\\Local\\Google\\Chrome\\User Data\\Default") 
    driver = webdriver.Chrome(options=chrome_options)
    wait = WebDriverWait(driver, 10)

    # acessar o site
    driver.get("https://m.esportesdasorte.com/ptb")
    sleep(10)

    erro = False    
    try: 
        efetuar_login(wait)
        sleep(2)

        lista_campeonato = ['brasileirao', 'libertadores']

        registros_resultado = []
        registros_dupla_chance = []

        for campeonato in lista_campeonato:
            try:
                if (campeonato == 'brasileirao'):
                    driver.get("https://m.esportesdasorte.com/ptb/bet/sports/soccer/brazil")
                elif (campeonato == 'libertadores'):                    
                    driver.get("https://m.esportesdasorte.com/ptb/bet/sports/soccer/international-clubs")
                    
                wait = WebDriverWait(driver, 10)

                for i in range(2):
                    if i == 1: 
                        if (campeonato == 'brasileirao'):
                            dupla_chance = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="allCnt"]/main/place-bet/div/sports/main/app-season-list/div/app-nested-view/div/div/app-btg-list/ul/li[5]/a')))

                        elif (campeonato == 'libertadores'):
                            dupla_chance = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="allCnt"]/main/place-bet/div/sports/main/app-season-list/div/app-nested-view/div/div/app-btg-list[1]/ul/li[5]/a')))

                        dupla_chance.click()
                    
                    sleep(4)
                    divs_resultado = driver.find_elements(by=By.CLASS_NAME, value='match-content')
                    for div in divs_resultado:
                        # div.text = 'Vitória BA \n Goianiense \n 18/05 16:00 \n BB \n +354 \n equalizer \n 1.94 \n 3.34 \n 4.45'
                        infos = div.text.split('\n')
                        time_casa = infos[0]
                        time_visitante = infos[1]
                        data = infos[2]

                        odd_casa = ""
                        odd_empate = ""
                        odd_fora = ""
                        odd_1x = ""
                        odd_12 = ""
                        odd_x2 = ""

                        if i == 0:
                            odd_casa = infos[len(infos)-3]
                            odd_empate = infos[len(infos)-2]
                            odd_fora = infos[len(infos)-1]
                            registros_resultado.append([campeonato, time_casa, time_visitante, data, odd_casa, odd_empate, odd_fora])
                        else: 
                            odd_1x = infos[len(infos)-3]
                            odd_12 = infos[len(infos)-2]
                            odd_x2 = infos[len(infos)-1]
                            registros_dupla_chance.append([campeonato, time_casa, time_visitante, data, odd_1x, odd_12, odd_x2])
            except:
                continue

        driver.quit() 
                
        df_resultado = pd.DataFrame(registros_resultado, columns=['campeonato', 'time_casa', 'time_visitante', 'data', 'odd_casa', 'odd_empate', 'odd_visitante'])
        df_dupla_chance = pd.DataFrame(registros_dupla_chance, columns=['campeonato', 'time_casa', 'time_visitante', 'data', 'odd_1x', 'odd_12', 'odd_x2'])
        new_df = pd.concat([df_resultado, df_dupla_chance[['odd_1x', 'odd_12', 'odd_x2']]], axis=1)
        new_df.to_csv(caminho_arquivo, index=False, sep=';', encoding='utf-8')

        print('capturado')
    except: 
        erro = True
        driver.quit()

def apostar_esportedasorte():
    file = 'bilhetes.csv'
    df = pd.read_csv(file, delimiter=';')
    df.set_index('Numero', inplace=True)

    chrome_options = Options()                   
    chrome_options.add_argument("--user-data-dir=C:\\Users\\mauri\\AppData\\Local\\Google\\Chrome\\User Data\\Default") 
    driver = webdriver.Chrome(options=chrome_options)
    wait = WebDriverWait(driver, 10)

    # acessar o site
    driver.get("https://m.esportesdasorte.com/ptb")
    sleep(10)

    erro = False
    try:
        efetuar_login(wait)
        sleep(2)

        try:                                                  
            apostas = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="bslpb"]/span/small')))
            if (int(apostas.text) > 0):
                apostas.click()
                if (int(apostas.text) == 1):
                    apagar = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="single-betslip"]/div[2]/div[2]/div[2]/a[1]')))
                else:
                    apagar = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="multi-betslip"]/div[2]/div[2]/div[3]/a[1]')))
                apagar.click()
        except:
            print('sem apostas')

        previous_campeonato = None
        previous_index = 1
        valor_aposta = '0'
        qtd_jogos = 0
        skipou = False
        for index, row in df.iterrows():
            if row['Apostado'] == 1:
                previous_index = index
                skipou = True
                continue

            qtd_jogos += 1
            if (index != previous_index) and (not skipou):
                apostar(wait, valor_aposta, qtd_jogos == 1)
                qtd_jogos = 0

            previous_index = index
            
            skipou = False
            
            valor_aposta = str(row['Aposta'])
            print('valor_aposta', valor_aposta)
            
            if (row['Campeonato'] == 'brasileirao'):
                # acessar o site
                if previous_campeonato != row['Campeonato']:
                    driver.get("https://m.esportesdasorte.com/ptb/bet/sports/soccer/brazil")
                    wait = WebDriverWait(driver, 10)
                else: 
                    resultado = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="allCnt"]/main/place-bet/div/sports/main/app-season-list/div/app-nested-view/div/div/app-btg-list/ul/li[1]/a')))
                    driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'nearest'});", resultado)
                    driver.execute_script("window.scrollBy(0, -100);")
                    resultado.click()
            elif (row['Campeonato'] == 'libertadores'):
                # acessar o site
                if previous_campeonato != row['Campeonato']:
                    driver.get("https://m.esportesdasorte.com/ptb/bet/sports/soccer/international-clubs")
                    wait = WebDriverWait(driver, 10)
                else: 
                    resultado = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="allCnt"]/main/place-bet/div/sports/main/app-season-list/div/app-nested-view/div/div/app-btg-list[1]/ul/li[1]/a')))
                    driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'nearest'});", resultado)
                    driver.execute_script("window.scrollBy(0, -100);")
                    resultado.click()
            
            sleep(1)

            print('passou do campeonato')
            
            previous_campeonato = row['Campeonato']

            for i in range(2):
                if i == 1: 
                    if (row['Campeonato'] == 'brasileirao'):
                        dupla_chance = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="allCnt"]/main/place-bet/div/sports/main/app-season-list/div/app-nested-view/div/div/app-btg-list/ul/li[5]/a')))

                    elif (row['Campeonato'] == 'libertadores'):
                        dupla_chance = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="allCnt"]/main/place-bet/div/sports/main/app-season-list/div/app-nested-view/div/div/app-btg-list[1]/ul/li[5]/a')))

                    driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'nearest'});", dupla_chance)                 
                    dupla_chance.click()

                parar = False
                sleep(4)
                divs_resultado = driver.find_elements(by=By.CLASS_NAME, value='match-content')
                for div in divs_resultado:
                    # div.text = 'Vitória BA \n Goianiense \n 18/05 16:00 \n BB \n +354 \n equalizer \n 1.94 \n 3.34 \n 4.45'
                    infos = div.text.split('\n')
                    time_casa = infos[0]
                    time_visitante = infos[1]
                    data = infos[2]

                    jogo = str.split(row['Jogo'], ' x ')
                    dftime_casa = jogo[0]
                    dftime_visitante = jogo[1]
                    dfdata = row['Data']

                    if (time_casa == dftime_casa and
                        time_visitante == dftime_visitante and
                        data == dfdata):
                        elementos_internos = div.find_elements(By.TAG_NAME, 'a')  
                        if (i == 0 and pd.isna(row['Resultado2'])):
                            if (row['Resultado1'] == time_casa):                            
                                driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'nearest'});", elementos_internos[1])
                                driver.execute_script("window.scrollBy(0, 100);")
                                elementos_internos[1].click()
                                parar = True
                                break
                            elif (row['Resultado1'] == time_visitante): 
                                driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'nearest'});", elementos_internos[3])
                                driver.execute_script("window.scrollBy(0, 100);")
                                elementos_internos[3].click()
                                parar = True
                                break
                            elif (row['Resultado1'] == 'Empate'):
                                driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'nearest'});", elementos_internos[2])
                                driver.execute_script("window.scrollBy(0, 100);")
                                elementos_internos[2].click()
                                parar = True
                                break
                        elif (i == 1 and not pd.isna(row['Resultado2'])):
                            if (row['Resultado1'] == time_casa and row['Resultado2'] == 'Empate'):
                                driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'nearest'});", elementos_internos[1])
                                driver.execute_script("window.scrollBy(0, 100);")
                                elementos_internos[1].click()
                                break
                            elif (row['Resultado1'] == 'Empate' and row['Resultado2'] == time_visitante):
                                driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'nearest'});", elementos_internos[3]) 
                                driver.execute_script("window.scrollBy(0, 100);")
                                elementos_internos[3].click()
                                break
                            elif (row['Resultado1'] == time_casa and row['Resultado2'] == time_visitante):
                                driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'nearest'});", elementos_internos[2])
                                driver.execute_script("window.scrollBy(0, 100);")
                                elementos_internos[2].click()
                                break

                if parar:
                    break

        apostar(wait, valor_aposta, qtd_jogos == 1)

        driver.quit() 

        print('apostado')
    except:
        erro = True
        driver.quit() 

def credito_esportedasorte(valor):
    chrome_options = Options()                   
    chrome_options.add_argument("--user-data-dir=C:\\Users\\mauri\\AppData\\Local\\Google\\Chrome\\User Data\\Default") 
    driver = webdriver.Chrome(options=chrome_options)
    wait = WebDriverWait(driver, 10)

    # acessar o site
    driver.get("https://m.esportesdasorte.com/ptb")
    sleep(10)

    erro = False     
    try: 
        efetuar_login(wait)
        sleep(2)

        icon_user = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="hdr-c"]/nav/div/span/a/i')))
        icon_user.click()

        depositar = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="user-menu"]/div[2]/ul/li[1]/ul/li/div/ul/li[1]/a')))
        depositar.click()
        
        sleep(1)
        driver.execute_script("window.scrollBy(0, 600);")
        sleep(1)

        paybrokers = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="allCnt"]/main/app-dashboard/payment-component/deposit-money/deposit-withdraw-money/div[2]/div/div/ul/li[1]/a')))
        paybrokers.click()

        input = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="mntNpt"]')))
        input.send_keys(valor)

        finalizar = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="allCnt"]/main/app-dashboard/payment-component/payment-form-out/payment-form/div[2]/div/div/form/div/div/button')))
        finalizar.click()
                                                         
        copiar = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="allCnt"]/main/app-dashboard/payment-component/payment-result/div/div/div/div[2]/div[4]/div/button')))
        copiar.click()
        
        qrcode = pyperclip.paste()
        print(qrcode)
        
        driver.quit()
        return qrcode        
    except:
        erro = True
        driver.quit()

def saldo_esportedasorte():
    chrome_options = Options()                   
    chrome_options.add_argument("--user-data-dir=C:\\Users\\mauri\\AppData\\Local\\Google\\Chrome\\User Data\\Default") 
    driver = webdriver.Chrome(options=chrome_options)
    wait = WebDriverWait(driver, 10)

    # acessar o site
    driver.get("https://m.esportesdasorte.com/ptb")
    sleep(10)

    erro = False     
    try: 
        efetuar_login(wait)
        sleep(2)

        icon_user = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="hdr-c"]/nav/div/span/a/i')))
        icon_user.click()

        texto_valor = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="user-menu"]/div[2]/div/div[4]/a[2]/span[2]')))
        valor = texto_valor.text.replace('R$ ', '')

        driver.quit()

        return valor
    except:
        erro = True
        driver.quit()
    
#credito_esportedasorte(10)

#apostar_esportedasorte()

app = Flask(__name__)

@app.route('/buscar')
def run_buscar():

    caputrar_esportedasorte()
    
    caminho_arquivo = "partidas.csv"

    if erro:
        return "Ocorreu um erro", 400

    return send_file(
        caminho_arquivo,
        mimetype='text/csv',
        as_attachment=True,
        download_name=caminho_arquivo
    )

@app.route('/apostar', methods=['POST'])
def run_apostar():
    if 'file' not in request.files:
        return "No file part in the request", 400
    file = request.files['file']
    if file.filename == '':
        return "No file selected", 400
    
    #Especifica o caminho do arquivo que deseja excluir
    caminho_arquivo = "bilhetes.csv"

    #Verifica se o arquivo existe antes de excluí-lo
    if os.path.exists(caminho_arquivo):
        os.remove(caminho_arquivo)
        print(f"O arquivo {caminho_arquivo} foi excluído com sucesso")
    else:
        print(f"O arquivo {caminho_arquivo} não existe")

    df = pd.read_csv(file, delimiter=';')
    df.to_csv(caminho_arquivo, sep=';', index=False)  # Salva o DataFrame sem a coluna de índice

    apostar_esportedasorte()

    if erro:
        return "Ocorreu um erro", 400

    return "Apostas realizadas!"

@app.route('/credito')
def run_credito():
    if 'valor' not in request.args:
        return "No 'valor' in the request", 400
    valor = request.args['valor']   

    result = credito_esportedasorte(valor)

    if erro:
        return "Ocorreu um erro", 400
    
    return result

@app.route('/saldo')
def run_saldo():
    result = saldo_esportedasorte()

    if erro:
        return "Ocorreu um erro", 400
    
    return result

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5032, debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5032
 * Running on http://192.168.1.15:5032
Press CTRL+C to quit


O arquivo partidas.csv foi excluído com sucesso
logado
capturado


127.0.0.1 - - [06/Jul/2024 01:48:00] "GET /buscar HTTP/1.1" 200 -


O arquivo partidas.csv foi excluído com sucesso
logado
capturado


...

127.0.0.1 - - [10/Jul/2024 00:28:58] "GET /credito?valor=20.0 HTTP/1.1" 200 -


logado


127.0.0.1 - - [10/Jul/2024 00:30:11] "GET /saldo HTTP/1.1" 200 -


O arquivo bilhetes.csv foi excluído com sucesso
logado
sem apostas
valor_aposta 20
passou do campeonato
valor_aposta 20
passou do campeonato
valor_aposta 20
passou do campeonato
valor_aposta 20
passou do campeonato
valor_aposta 20
passou do campeonato
valor_aposta 20
passou do campeonato
valor_aposta 20
passou do campeonato
valor_aposta 20
passou do campeonato
apostado


127.0.0.1 - - [10/Jul/2024 00:36:09] "POST /apostar HTTP/1.1" 200 -
